Question we want to ask:

When is central-place foraging likely to emerge relative to point-to-point?



Variables we are interested in testing for the emergence of CPF: (I am writing all)

1. Sleeping sites distribution
2. Resource type targeted (big/small game)
3. Information-sharing
4. Inter-forager correlation (how spatially correlated foragers are)
5. Intra-forager variation (how successful a forager is)
6. Resource sharing (between families in the same camp)
7. Ability to rest (maybe it can be folded within resource sharing)
8. Ability to defend against predators 



Our hypotheses:

When is CPF better: when temporal, spatial, skill synchrony between foragers is beneficial, when there's a limited availability of sleeping sites, when there are multiple-co depending offspring, when resources are unpredictable and hard to find.

When is P2P better: `**we need to spell out the potential costs of CPF and how to simulate them so that in some instances P2P is better**`




## Define environmental variables


class environment:


##### global variables

n_sleep_site = number of sleeping sites 

n_agents = number of foragers

p_sharing = [0.1, 0.5, 0.9] - probability that an agent shares the harvested with others in their location at the end of the day

n_large_res = number of large resources

n_small_res = number of small resources

pref_previous = y - some factor by which we multiply p_sharing to scale it depending on how many interactions

p_info = [0.1, 0.5, 0.9] - probability that an agent shares information about location of resources with another agent

mov_cost = cost of moving 1km 

min_carrying = minimum amount of energy an agent must have in order to carry a resource instead of consuming it

##### resources 

type = small, large

location = xy coordinates of resource

prob_find = [0.1, 0.5] - probability to find each resource alone, unskilled

prob_harvest = [0.1, 0.5] - probability to harvest each resource alone 

prop_energy = [0.1, 0.5] - proportion of daily requirement satisfied by each resource 


#### sleeping sites

location = xy coordinates of sleeping site

max_people = maximum number of foragers that can reside in a sleeping site



## Agent variables

Agents are a family. E requirement of a family = 7,500kcal

location = xy coordinates of agent

carrying = number of resources agent is carrying

for_speed = [3, 5] foraging speed - how many kilometers an agent can travel in 1h. 

max_carry = maximum amoung of resources that the agent can carry (will be the equivalent of 2 large resources or 4 small ones?)

foraging distance (d) = [5, 10, 20] (km) how far the agent goes to find resources (maximum). small d will mean that they stay close to sleeping site and only find easily available resources like fruits. 

energy (e) : updated each day 

information (i): information for each resource = [0.1, 0.2, ...]. this will update whenever agent finds a resource and when it learns from someone else 

home (p_h): [x,x] tendency to return to sleeping site where they were the previous night (home i_d)

#inter-forager correlation (c): [] length will be equal to num of agents (CP: i'd leave this for stage 2)

p_f = tendency_to_forage: what proportion of days the agent goes foraging (can also be equivalent to what proportion of the members of the family goes foraging (we ignore inter-forager difference in skill?)


each day:

1. agent starts at a sleeping site
2. it forages (8h/day foraging or moving)
    - Agent starts foraging within d
    - if a resource is found
        - Resource is harvested with probability prob_harvested * i (information) of the agent over that resource
    - pay cost of moving depending on how many km
    - if resource is harvested
        - agent carries the resource/consumes the resource depending on whether its energy is higher than min_carrying
    - if agent is carrying less resources than max_carrying & has been foraging less than 8h
        - agent continues foraging

3. end of day: find sleeping site: 
    - pick a new site vs. one from previous day according to p_h
    - if the site from previous day picked, 
        -   pay cost of moving back 
    - else
        -   pay cost of moving to next available sleeping site

4. at sleeping site:
    - count how many foragers at that site 
    - share resources with others according to p_sharing and pref_previous
    - share information with others with some probability (global variable)

## Reporters

Make plots of the following variables over time (over the course of the simulation)

- Mean e of all agents (over time)
- Variance in e of agents

Make plots of the following variables at the end of the simulation: 

- Number of agents alive
- Time til population collapses?

In [ ]:
import random
import numpy as np
import copy

# Implementation
## Global variables

In [ ]:
# Define the playing field: let it be a 20x20 grid -- 400 positions
GRID_SIZE = 20
grid = [[None for x in range(GRID_SIZE)] for y in range(GRID_SIZE)]
N_LOCATIONS = GRID_SIZE * GRID_SIZE

# Define the global variables
N_SLEEP_SITES = N_LOCATIONS / 10
N_LARGE_RES = N_LOCATIONS / 40
N_SMALL_RES = N_LARGE_RES * 4 # Four times as many as large resources


# Define probabilities
P_SHARING = 0.3 # Can be 0.1, 0.5 or 0.9, probability of sharing harvested resources
P_INFO = 0.3 # Can be 0.1, 0.5 or 0.9, probability of sharing information
PREF_PREVIOUS = 1.2 # Increase of sharing harvest/info probability with agents previously shared with
PREF_PREVIOUS_MOVE = 0.7 # Probability of moving along the same direction as the previous move

# Define costs
MOVE_COST = 80 # Cost of moving 1 km in kcal (Google -- 1 km per kilogram of weight)
CARRY_COST = 10 # Minimum energy in kcal needed to carry 1 unit of resource instead of consuming it on the spot (Around 10 kg -- 10 kcal)

## Classes 

In [ ]:
# Define some constants
SMALL_RES = 'small_res'
LARGE_RES = 'large_res'

ENERGY_REQUIREMENT = 7500 # Energy requirement for a day, in kcal

HIGH_PROB = 0.7
MEDIUM_PROB = 0.5
LOW_PROB = 0.3

def distance(loc1: np.ndarray, loc2: np.ndarray):
    """Calculate the Manhattan distance between two locations."""
    return abs(loc1[0] - loc2[0]) + abs(loc1[1] - loc2[1])

def resource_type_weighted_distance(loc1: np.ndarray, loc2: np.ndarray, res_type):
    """Calculate the Manhattan distance between two locations, weighted by the type of resource.
    
    Distance is "twice as small" for large resources compared to small resources."""
    return (abs(loc1[0] - loc2[0]) + abs(loc1[1] - loc2[1])) * (1 if res_type == SMALL_RES else 1/2)

# Define necessary classes for the game/project/simulation

class Resource():
    def __init__(self, res_location: np.ndarray):
        self.location: np.ndarray = res_location
        
        # Flip a coin for res_type, p_find, p_harvest, p_energy
        self.res_type = SMALL_RES if random.random() < 0.5 else LARGE_RES
        if self.res_type == SMALL_RES:
            self.p_find = LOW_PROB if random.random() < 0.5 else HIGH_PROB # Probability of finding the resource, unskilled
            self.p_harvest = MEDIUM_PROB if random.random() < 0.5 else HIGH_PROB # Probability of harvesting the resource, alone
            self.prop_energy = 0.1 if random.random() < 0.5 else 0.5 # Proportion of daily energy consuming the resource provides
        elif self.res_type == LARGE_RES:
            self.p_find = MEDIUM_PROB if random.random() < 0.5 else HIGH_PROB # Probability of finding a large resource is higher
            self.p_harvest = LOW_PROB if random.random() < 0.5 else MEDIUM_PROB # Probability of harvesting a large resource is lower
            self.prop_energy = 0.7 if random.random() < 0.5 else 0.9 # Large resource provides more energy

        self.res_weight = 1 if self.res_type == SMALL_RES else 2 # Weight of the resource

    def search_for_resource(self):
        """Search for the resource at the location. Returns True if the agent finds the resource, False otherwise."""
        # Roll a dice to see if the agent can find the resource
        if random.random() < self.p_find:
            return True
        return False

    def harvest(self):
        """Harvest the resource at the location. Returns a HarvestedResource object if the agent manages to harvest the resource."""
        # Roll a dice to see if the agent can harvest the resource
        if random.random() < self.p_harvest:
            return HarvestedResource(self.res_type, self.prop_energy)

class HarvestedResource():
    def __init__(self, res_type, prop_energy):
        self.res_type = res_type
        self.res_weight = 1 if self.res_type == SMALL_RES else 2 # Weight of the resource
        self.energy = prop_energy

    def consume(self):
        """Consume the resource. Returns the energy provided by the resource."""
        return self.energy


# TODO: If an agent doesn't find a resource, coming from knowledge from another agent, the probability of sharing with that agent reduces

class Agent():
    def __init__(self, agent_id: int, agent_location: np.ndarray):
        self.id = agent_id # Unique identifier
        self.location: np.ndarray = agent_location # Current location
        self.energy = 0 # Energy level

        self.harvested_resources: list[HarvestedResource] = []
        self.capacity = 4 # Maximum number of resource weight the agent can carry
        self.knowledge: list[Resource] = [] # info of resource locations

        self.last_move_direction = None # Last move direction
        self.last_sleep_site = agent_location # Last sleep site

        self.desired_move_location = None # Desired move location
        self.desired_loc_reached = False # Whether the desired location was reached

        # Health reduces by percent of energy requirement not met
        # and increases by percent of surplus energy percentage.
        # If health is below 50, the agent has lower probability of finding/harvesting resources.
        # If health reaches 0, the agent dies.
        self.health = 100 # Health level
        self.dead = False # Dead or alive

        # Flip a coin for move_speed
        self.move_speed = 3 if random.random() < 0.5 else 5 # Speed of the agent in km/h

        # Flip a coin for maximum distance willing to be far from last sleep site
        max_forage_dist_flip = random.random()
        if max_forage_dist_flip < 1/3:
            self.max_forage_dist = 5 
        elif max_forage_dist_flip < 2/3:
            self.max_forage_dist = 10
        else:
            self.max_forage_dist = 20 #TODO

        # TODO: Need some sort of memory?
    
    def daily_reset(self):
        """Reset the agent's energy level and calculate health level at the start of next day."""
        if self.energy > ENERGY_REQUIREMENT:
            self.health += (self.energy - ENERGY_REQUIREMENT) / ENERGY_REQUIREMENT
            # Health cannot exceed 100
            if self.health > 100:
                self.health = 100

        elif self.energy < ENERGY_REQUIREMENT:
            self.health -= (ENERGY_REQUIREMENT - self.energy) / ENERGY_REQUIREMENT
        
        if self.health < 0:
            self.dead = True
        
        self.energy = 0
        self.last_move_direction = None
    
        self.desired_loc_reached = False
        # With a probability 0.7, choose a new desired location, otherwise, do random foraging walks
        if len(self.knowledge) > 0 and random.random() < 0.7:
            # Sort the knowledge by weighted distance to the agent
            self.knowledge.sort(key=lambda x: resource_type_weighted_distance(self.location, x.location, x.res_type))

            # Choose the closest resource location which is inside self.max_forage_dist
            for res in self.knowledge:
                if distance(self.location, res.location) <= self.max_forage_dist:
                    self.desired_move_location = res.location
                    break
        else:
            self.desired_move_location = None
        
        
    def daily_foraging(self, new_location):
        """Execute the daily 8-hour foraging of the agent."""
        steps_left_to_take = self.move_speed * 8

        while steps_left_to_take > 0:
            steps_left_to_take -= 1

            # If the agent has a desired location and it hasn't been reached yet, move towards it
            if self.desired_move_location is not None and not self.desired_loc_reached:
                # Move towards the desired location
                self.move_towards_location(self.desired_move_location)
                
                # Check if the desired location was reached
                if self.location == self.desired_move_location:
                    self.desired_loc_reached = True

            else:
                # Move randomly
                self.move_randomly()
                steps_left_to_take -= 1
            
            # Forage the current location
            self.forage_location()

            # TODO:



    def move_towards_location(self, desired_location):
        """Move towards the desired location."""
        # Calculate the direction to move
        move_direction = desired_location - self.location
        previous_location = copy.deepcopy(self.location)

        # Do one step
        if move_direction[0] == 0:
            # Move along the y-axis if the x-axis is reached
            self.location[1] += np.sign(move_direction[1])
        elif move_direction[1] == 0:
            # Move along the x-axis if the y-axis is reached
            self.location[0] += np.sign(move_direction[0])
        else:
            # Prefer moving along the same direction as the last move
            if self.last_move_direction is not None and random.random() < PREF_PREVIOUS_MOVE:
                # Move along the same direction as the last move
                self.location += self.last_move_direction
            else:
                # Move along the desired direction
                if random.random() < 0.5:
                    # Move along the x-axis
                    self.location[0] += np.sign(move_direction[0])
                else:
                    # Move along the y-axis
                    self.location[1] += np.sign(move_direction[1])

        # Update the last move direction
        self.last_move_direction = self.location - previous_location
    
    def move_randomly(self):
        """Move randomly."""
        # Randomly choose a direction to move, prefer moving along the same direction as the last move
        if self.last_move_direction is not None and random.random() < PREF_PREVIOUS_MOVE:
            # Move along the same direction as the last move
            self.location += self.last_move_direction
        else:
            # Move randomly
            move_direction = np.random.choice([np.array([1, 0]), np.array([-1, 0]), np.array([0, 1]), np.array([0, -1])])
            self.location += move_direction
            self.last_move_direction = move_direction
        



    def consume_resouce(self):
        # Consume the resource
        self.energy += self.prop_energy * ENERGY_REQUIREMENT
        # TODO: not finished
        
    def forage_location(self):
        # TODO: not finished
        